# Code lấy dữ liệu từ BaseVN

## 1. Kết nối notebook với Google Drive
Bước này có thể bỏ qua nếu không muốn lưu vào Drive. Tuy nhiên các output files sẽ mất sau khi restart session

In [2]:
"""
Chạy đoạn code dưới đây để kết nối notebook với Google Drive
"""
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Gửi request và lấy dữ liệu form

In [5]:
import requests
import pandas as pd
import os
import json
from tqdm import tqdm
import numpy as np

def get_data(url: str = "https://workflow.base.vn/extapi/v1/workflow/jobs",
             payload: dict = {}):

    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }

    response = requests.post(url, data=payload, headers=headers)
    print("Status code:", response.status_code)
    try:
        return response.json()
    except ValueError:
        return {}


if __name__ == "__main__":
    access_token = "ADD YOUR TOKEN HERE"
    output_folder_path = "/content/drive/MyDrive/outputs" # Thay output path bằng folder chứa file kết quả

    # Sửa đổi payload các trường cần lấy data theo format {"key1": "value1", "key2": "value2", ...}
    payload = {
        "access_token": f"{access_token}",
        "id": "13080",
        "created_from": "8/4/2025",
        "created_to": "8/4/2025"
    }
    data = get_data(payload=payload)
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    full_values = []
    for idx, job in tqdm(enumerate(data["jobs"])):
        form = job["form_origin"]
        headers, values = [], []
        for item in form:
            # print(item)
            name = item['name']
            value = item['value']
            headers.append(name)
            values.append(value)
        full_values.append(values)
        print(len(headers))
    df = pd.DataFrame(full_values, columns=headers)

    # full_output.xlsx là tên file cần lưu
    df.to_excel(f'{output_folder_path}/full_output.xlsx', index=False)

Status code: 200


3it [00:00, 5456.60it/s]

45
45
45
